# Task 5: Transferring Attacks
Evaluate the classification accuracy (Top-1 and Top-5) on a DenseNet121 model for Clean and all `.pt` adversarial datasets in the current directory.

In [1]:
import os, json, torch, numpy as np
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets, transforms, models
from pathlib import Path

@torch.no_grad()
def accuracy(model, loader, idx2true, device):
    t1 = t5 = n = 0
    for x, labs in loader:
        x = x.to(device, non_blocking=True)
        y = torch.tensor([idx2true[int(l)] for l in labs], device=device)
        out = model(x)
        t1 += (out.argmax(1) == y).sum().item()
        t5 += (out.topk(5, 1)[1] == y[:, None]).any(1).sum().item()
        n += y.size(0)
    return 100 * t1 / n, 100 * t5 / n

def adv_loader(path, batch_size, workers):
    ckpt = torch.load(path, map_location="cpu")
    return DataLoader(TensorDataset(ckpt["images"], ckpt["labels"]),
                      batch_size=batch_size, num_workers=workers)

In [2]:
# Config
data_path = "./TestDataSet"
label_path = "./TestDataSet/labels_list.json"
batch_size = 32
num_workers = 4
model_name = "densenet121"

# Device and transforms
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean.tolist(), std.tolist())
])

# Load clean dataset and label map
clean_ds = datasets.ImageFolder(data_path, transform=transform)
clean_ld = DataLoader(clean_ds, batch_size=batch_size, num_workers=num_workers)
with open(label_path) as f:
    idx2true = {i: int(e.split(":", 1)[0]) for i, e in enumerate(json.load(f))}

In [3]:
# Load model
mdl_ctor = getattr(models, model_name)
mdl = mdl_ctor(weights="IMAGENET1K_V1").to(device)
mdl.eval()

# Load all .pt files in current directory
loaders = {
    "Clean": clean_ld
}
for ptfile in sorted(Path(".").glob("*.pt")):
    label = ptfile.stem
    loaders[label] = adv_loader(str(ptfile), batch_size, num_workers)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /home/sj4025/.cache/torch/hub/checkpoints/densenet121-a639ec97.pth
100%|██████████| 30.8M/30.8M [00:00<00:00, 106MB/s] 


In [4]:
# Evaluate all
print(f"\nTransfer accuracy on {model_name} ({batch_size}-batch)")
print("Dataset                               Top‑1   Top‑5")
print("-" * 45)
for name, ld in loaders.items():
    a1, a5 = accuracy(mdl, ld, idx2true, device)
    print(f"{name:<35} {a1:6.2f}% {a5:6.2f}%")


Transfer accuracy on densenet121 (32-batch)
Dataset                               Top‑1   Top‑5
---------------------------------------------
Clean                                74.60%  93.60%
FGSM                                 45.40%  75.80%
PGD-Step-10                          48.00%  82.20%
PGD-Step-15                          49.60%  82.60%
PGD-Step-5                           45.80%  79.20%
PGD-Targeted-Fixed-Step-10           72.20%  91.80%
PGD-Targeted-Fixed-Step-15           73.20%  92.60%
PGD-Targeted-Fixed-Step-5            72.00%  90.40%
PGD-Targeted-Step-10                 71.80%  91.80%
PGD-Targeted-Step-15                 72.80%  93.00%
PGD-Targeted-Step-5                  71.00%  89.60%
adv_set_patch_soft_eps0.3_step110    73.00%  92.40%
adv_set_patch_soft_eps0.3_step50     71.00%  91.60%
adv_set_patch_soft_eps0.3_step80     71.60%  92.60%
adv_set_patch_soft_eps0.3_step80_untargeted  68.80%  91.00%
adv_set_patch_soft_eps0.5_step80     72.00%  91.40%
adv_set_patch_sof